## load packages

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.feature_selection import VarianceThreshold

import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Input

## Preprocess the data


In [3]:
file_path = "/content/drive/MyDrive/DTSC Big Data/BigData/amazon_reviews_us_Electronics_v1_00.tsv" #bh amazon electronics data file path
data = pd.read_csv(file_path,  sep='\t', on_bad_lines='skip') # load and read dataset
data.describe() # summary data

,customer_id,product_parent,star_rating,helpful_votes,total_votes
count,3.091024e+06,3.091024e+06,3.091024e+06,3.091024e+06,3.091024e+06
mean,2.879196e+07,5.102057e+08,4.035415e+00,1.860874e+00,2.372440e+00
std,1.542984e+07,2.868327e+08,1.387460e+00,2.133715e+01,2.249700e+01
min,1.000500e+04,6.478000e+03,1.000000e+00,0.000000e+00,0.000000e+00
25%,1.503881e+07,2.623618e+08,3.000000e+00,0.000000e+00,0.000000e+00
50%,2.806679e+07,5.085523e+08,5.000000e+00,0.000000e+00,0.000000e+00
75%,4.328128e+07,7.632433e+08,5.000000e+00,1.000000e+00,1.000000e+00
max,5.309658e+07,9.999982e+08,5.000000e+00,1.278600e+04,1.294400e+04


In [4]:
data.head() # preview dataset

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,41409413,R2MTG1GCZLR2DK,B00428R89M,112201306,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,Electronics,5,0,0,N,Y,Five Stars,As described.,2015-08-31
1,US,49668221,R2HBOEM8LE9928,B000068O48,734576678,"Hosa GPM-103 3.5mm TRS to 1/4"" TRS Adaptor",Electronics,5,0,0,N,Y,It works as advertising.,It works as advertising.,2015-08-31
2,US,12338275,R1P4RW1R9FDPEE,B000GGKOG8,614448099,Channel Master Titan 2 Antenna Preamplifier,Electronics,5,1,1,N,Y,Five Stars,Works pissa,2015-08-31
3,US,38487968,R1EBPM82ENI67M,B000NU4OTA,72265257,LIMTECH Wall charger + USB Hotsync & Charging ...,Electronics,1,0,0,N,Y,One Star,Did not work at all.,2015-08-31
4,US,23732619,R372S58V6D11AT,B00JOQIO6S,308169188,Skullcandy Air Raid Portable Bluetooth Speaker,Electronics,5,1,1,N,Y,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31


In [5]:
data.info() # columns and datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3091024 entries, 0 to 3091023
Data columns (total 15 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   marketplace        object
 1   customer_id        int64 
 2   review_id          object
 3   product_id         object
 4   product_parent     int64 
 5   product_title      object
 6   product_category   object
 7   star_rating        int64 
 8   helpful_votes      int64 
 9   total_votes        int64 
 10  vine               object
 11  verified_purchase  object
 12  review_headline    object
 13  review_body        object
 14  review_date        object
dtypes: int64(5), object(10)
memory usage: 353.7+ MB


In [6]:
data['product_id'].value_counts()[:15]

B003L1ZYYM    15334
B0002L5R78    11166
B003EM8008     9766
B0012S4APK     9359
B0001FTVEK     8793
B000WYVBR0     7835
B0019EHU8G     7586
B00F5NE2KG     6688
B004QK7HI8     6536
B00D5Q75RC     6062
B00316263Y     5813
B0052SCU8U     5756
B001GTT0VO     5580
B002MAPT7U     5295
B003WGRUQQ     5072
Name: product_id, dtype: int64

In [7]:
review=pd.DataFrame(data.groupby('star_rating').size().sort_values(ascending=False).rename('No of reviews').reset_index())
review.head()

,star_rating,No of reviews
0,5,1779371
1,4,536417
2,1,357817
3,3,238387
4,2,179032


# Feature extractor

## select review_body coloum

In [8]:
data_n = data[['star_rating' , 'review_body' , 'review_headline' , 'customer_id']]
data_n=data_n.dropna()
data_n.head()

,star_rating,review_body,review_headline,customer_id
0,5,As described.,Five Stars,41409413
1,5,It works as advertising.,It works as advertising.,49668221
2,5,Works pissa,Five Stars,12338275
3,1,Did not work at all.,One Star,38487968
4,5,Works well. Bass is somewhat lacking but is pr...,Overall pleased with the item,23732619


In [9]:
check =  data_n[data_n["review_body"].isnull()]
check.head()

,star_rating,review_body,review_headline,customer_id


In [10]:
data_n = data_n.sample(n=10000) # randomly select 10000 rows of data as sample
print(len(data_n))

10000


In [11]:
actualrating = data_n[(data_n['star_rating'] == 1) | (data_n['star_rating'] == 5)]
actualrating.shape

(6933, 4)

In [12]:
y = actualrating['star_rating']
x = actualrating['review_body'].reset_index()

In [13]:
len(y)

6933

In [14]:
X = x['review_body']
print(X)

0       I listened to and examined this dock thoroughl...
1       Great value! Using on sat tv receivers at our ...
2       I purchased this for the rare occasions the my...
3       I bought these headphones a few months ago and...
4       Tough to give a \\"glowing\\" review for cable...
                              ...                        
6928                                    great clock radio
6929                             FAST SHIPPING,GOOD DEAL!
6930                                         Works great!
6931    I would highly recommend this product.  Inexpe...
6932    I absolutely love de ADAPT, I´ve been using it...
Name: review_body, Length: 6933, dtype: object


In [15]:
print(len(X))


6933


## Continuing Normalization

In [16]:
import string
from nltk.corpus import stopwords
# stop=set(stopwords.words('english'))
def text_process(text):
    '''
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Return the cleaned text as a list of words
    '''
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [17]:
tokens = X[0].split()
print(tokens)

['I', 'listened', 'to', 'and', 'examined', 'this', 'dock', 'thoroughly', 'and', 'was', 'not', 'impressed.', 'I', 'can', 'conclude', 'that', 'it', 'is', 'just', 'another', 'over-priced', 'and', 'over-hyped', 'item', 'like', 'so', 'many', 'that', 'are', 'currently', 'dominating', 'consumer', 'electronics.', 'First,', 'it', 'may', 'sound', 'decent', 'to', 'you,', 'but', "it's", 'not', 'worth', 'the', 'money,', 'I', 'have', 'heard', 'real', 'stereo', 'speakers', 'and', 'the', 'difference', 'is', 'night', 'and', 'day', 'even', 'to', 'my', 'old', 'JBLs', 'from', 'the', '70s.', 'The', 'Beosound', '8', 'sounds', 'muddy,', 'the', 'bass', 'is', 'limited', 'because', 'of', 'the', 'small', 'and', 'cheaper', 'woofer,', 'and', 'the', 'highs', 'lack', 'the', 'clarify', 'they', 'should', 'have', 'at', 'their', 'price', 'point.', 'Overall,', "it's", 'just', 'another', 'muddy', 'sounding', 'dock.', 'See', 'and', 'hear', 'for', 'yourself!', 'Start', 'by', 'looking', 'at', 'the', 'speakers', 'grills,', 't

In [18]:
sample_text = "Hey there! This is a sample review, which happens to contain punctuations."
print(text_process(sample_text))


['Hey', 'sample', 'review', 'happens', 'contain', 'punctuations']


# Feature vectors

## Vectorization

similiar to the idea of Shingling: Converts a document into a set representation (Boolean vector)

Here we use SciKit Learn’s CountVectorizer. This model will convert a collection of text documents to a matrix of token counts.

For example: 

word count/message| Message 1	| Message 2 | Message 3 | ......... | Message n |
------------------|------------|-----------|----------|-----------|-----------|
Word 1 Count      | 0          | 1         | 0        | 0         | 2         |
Word 2 Count      | 1          | 0         | 0        | 0         | 0         |
Word 3 Count      | 1          | 2         | 1        | 1         | 1         |
............      | 1          | 1         | 0        | 1         | 0         |
Word n Count      | 1          | 1         | 1        | 0         | 1         |

Since there are so many messages, we can expect a lot of zero counts for the presence of that word in that document. Because of this, SciKit Learn will output a [Sparse Matrix](https://en.wikipedia.org/wiki/Sparse_matrix).

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
# return a vector array
bow_transformer = CountVectorizer(analyzer=text_process).fit(X)

 bow = bag of words

In [20]:
len(bow_transformer.vocabulary_)

21649

In [21]:
review_66 = X[66]

In [22]:
bow_66 = bow_transformer.transform([review_66])
bow_66


<1x21649 sparse matrix of type '<class 'numpy.int64'>'
	with 31 stored elements in Compressed Sparse Row format>

In [23]:
print(bow_66)

  (0, 988)	2
  (0, 1255)	2
  (0, 1948)	1
  (0, 2011)	1
  (0, 3269)	2
  (0, 3386)	1
  (0, 3565)	4
  (0, 3590)	1
  (0, 4190)	4
  (0, 4191)	1
  (0, 5866)	1
  (0, 6677)	1
  (0, 7419)	1
  (0, 7481)	1
  (0, 7983)	1
  (0, 8409)	1
  (0, 8519)	1
  (0, 8834)	1
  (0, 9940)	1
  (0, 11496)	1
  (0, 12178)	1
  (0, 12634)	1
  (0, 16383)	1
  (0, 17272)	1
  (0, 18197)	1
  (0, 18612)	1
  (0, 20308)	1
  (0, 20969)	1
  (0, 21135)	1
  (0, 21171)	1
  (0, 21452)	1


In [24]:
bow_66.getnnz() 

31

total number of the unique words in review body no. 66, after removing stopwords. 4 of them reqeat 2 times, and 2 of them reqeat 3 times.



In [25]:
print(bow_transformer.get_feature_names_out()[7935])

anniversary


In [26]:
X = bow_transformer.transform(X)

In [27]:
print('Shape of Sparse Matrix: ', X.shape)

Shape of Sparse Matrix:  (6933, 21649)


In [28]:
print('Amount of Non-Zero occurences: ', X.nnz)

Amount of Non-Zero occurences:  182747


In [29]:
sparsity = (100.0 * X.nnz / (X.shape[0] * X.shape[1]))
print('sparsity: {}'.format(sparsity))
#non zero/total

sparsity: 0.12175623652177144


after count, there about 12.4% of the entire metrix are zeros.

In [30]:
#Lets start training the model
from sklearn.model_selection import train_test_split
#using 30% of the data for testing, this will be revised once we do not get the desired accuracy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=46)

In [31]:
y_test.shape, X_test.shape ,X_train.shape ,y_train.shape


((2080,), (2080, 21649), (4853, 21649), (4853,))

# Fitting simple Classifier model to the training set

## Training in different classifier 

to find the best fitting classifier for this dataset

* Naive Bayes Classifier
* Support Vector Machine
* K Neighbors Classifier
* Gradient Boosting Classifier


### Naive Bayes Classifier
[Text Classification using Naive Bayes](http://www.inf.ed.ac.uk/teaching/courses/inf2b/learnnotes/inf2b-learn-note07-2up.pdf)

[Naive Bayes classifier](https://en.wikipedia.org/wiki/Naive_Bayes_classifier)



In [32]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)

MultinomialNB()

In [33]:
preds = nb.predict(X_test)

In [34]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, preds))
print('\n')
print(classification_report(y_test, preds))
nb.score(X_train, y_train)

[[ 161  204]
 [  32 1683]]


              precision    recall  f1-score   support

           1       0.83      0.44      0.58       365
           5       0.89      0.98      0.93      1715

    accuracy                           0.89      2080
   macro avg       0.86      0.71      0.76      2080
weighted avg       0.88      0.89      0.87      2080



0.9486915310117453

* precision = TP / TP + FP
* recall = TP / TP + FN
* S = 1-(1-s^r)b ---false positive
* S = (1-s^r)b ---false negative


### support vector machine 

SVM maps training examples to points in space so as to maximise the width of the gap between the two categories. New examples are then mapped into that same space and predicted to belong to a category based on which side of the gap they fall.

[SVMs](https://scikit-learn.org/stable/modules/svm.html#:~:text=Support%20vector%20machines%20(SVMs)%20are,than%20the%20number%20of%20samples.)

SVC for binary classification

The advantages of support vector machines are:

* Effective in high dimensional spaces.

* Still effective in cases where number of dimensions is greater than the number of samples.

In [35]:
from sklearn.svm import SVC
clf = SVC() # classfiyer
clf.fit(X_train, y_train) 
predsvm=clf.predict(X_test)

In [36]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, preds))
print('\n')
print(classification_report(y_test, predsvm))
predsvm=clf.predict(X_test)
clf.score(X_train,y_train)

[[ 161  204]
 [  32 1683]]


              precision    recall  f1-score   support

           1       0.93      0.31      0.47       365
           5       0.87      0.99      0.93      1715

    accuracy                           0.88      2080
   macro avg       0.90      0.65      0.70      2080
weighted avg       0.88      0.88      0.85      2080



0.9528126931794766

### KNeighborsClassifier

[Classifier implementing the k-nearest neighbors vote.](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

[Neighbors-based classification](hhttps://scikit-learn.org/stable/modules/neighbors.html#classification) ,Classification is computed from a simple majority vote of the nearest neighbors of each point.

The *k*-neighbors classification in KNeighborsClassifier is the most commonly used technique. The optimal choice of the value *k* is highly data-dependent: in general a larger *k* suppresses the effects of noise, but makes the classification boundaries less distinct.

In [37]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X, y) 

KNeighborsClassifier(n_neighbors=3)

In [38]:
predsknn=neigh.predict(X_test)

In [39]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, predsknn))
print('\n')
print(classification_report(y_test, predsknn))
neigh.score(X_train,y_train)

[[ 159  206]
 [  27 1688]]


              precision    recall  f1-score   support

           1       0.85      0.44      0.58       365
           5       0.89      0.98      0.94      1715

    accuracy                           0.89      2080
   macro avg       0.87      0.71      0.76      2080
weighted avg       0.88      0.89      0.87      2080



0.8924376674222131

### GradientBoostingClassifier

In [40]:
from sklearn.ensemble import GradientBoostingClassifier
model= GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
model.fit(X_train, y_train)
predicted= model.predict(X_test)

In [41]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, predicted))
print('\n')
print(classification_report(y_test, predicted))
model.score(X_train,y_train)

[[ 171  194]
 [  65 1650]]


              precision    recall  f1-score   support

           1       0.72      0.47      0.57       365
           5       0.89      0.96      0.93      1715

    accuracy                           0.88      2080
   macro avg       0.81      0.72      0.75      2080
weighted avg       0.86      0.88      0.86      2080



0.9202555120543994

# Predicting the Result

In [42]:
actualrating['review_body'].head()

2488359    I listened to and examined this dock thoroughl...
529318     Great value! Using on sat tv receivers at our ...
2004392    I purchased this for the rare occasions the my...
580669     I bought these headphones a few months ago and...
2717817    Tough to give a \\"glowing\\" review for cable...
Name: review_body, dtype: object

In [44]:
positive_review = actualrating['review_body'][529318]
positive_review

'Great value! Using on sat tv receivers at our hotel. Performs as well as the high dollar cables.'

In [45]:
positive_review_transformed = bow_transformer.transform([positive_review])
nb.predict(positive_review_transformed)[0]

5

In [46]:
clf.predict(positive_review_transformed)[0]

5

In [47]:
actualrating['review_body'].tail()

320569                                     great clock radio
864397                              FAST SHIPPING,GOOD DEAL!
481938                                          Works great!
2087809    I would highly recommend this product.  Inexpe...
1231919    I absolutely love de ADAPT, I´ve been using it...
Name: review_body, dtype: object

In [48]:
positive_review = actualrating['review_body'][2087809]
positive_review

"I would highly recommend this product.  Inexpensive and is working great.  Now again how it holds out over a long period of time.  I'm thinking a couple of years down the road I don't know but for what it cost if it does stop working two years later not a bid deal."

In [49]:
positive_review_transformed = bow_transformer.transform([positive_review])
model.predict(positive_review_transformed)[0]

5

In [50]:
negative_review = data_n['review_body'][580669]
print(negative_review)

I bought these headphones a few months ago and they started to buzz so loud that I couldn't hear anything else. It was the noise cancelling feature, and it wouldn't stop even after I changed the battery. Not only that, but it suddenly snapped right before the hinge when I pulled them off! They are so cheap and flimsy that I pretty much gave up on them and bought some superior Bose headphones. However, I didn't want my $170 investment to go to waste, so I went through a runaround for WEEKS with Able Planet's poor excuse for customer service, and I was finally able to get them replaced!<br /><br />They shipped in yesterday, and the moment I pulled them onto my head... They snapped, right at the hinge.<br /><br />No more Able Planet for me. Do not buy from this awful company, it is so not worth the hassle from the bad product AND the bad customer service.


In [51]:
negative_review_transformed = bow_transformer.transform([negative_review])
nb.predict(negative_review_transformed)[0]

1

In [52]:
clf.predict(negative_review_transformed)[0]

1

# Visulazation

# Citation
* [Learning to Classify Text](https://www.nltk.org/book/ch06.html)
* [NLP with Python](https://rpubs.com/pjozefek/669929)
* [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
* [RBF SVM parameters](https://scikit-learn.org/stable/auto_examples/svm/plot_rbf_parameters.html)
* [Gaussian Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
* Géron, Aurélien, and Rebecca Demarest. “3. Classification/ Performance Measures/ Precision and Recall.” Hands-on Machine Learning with Scikit-Learn and Tensorflow: Concepts, Tools, and Techniques to Build Intelligent Systems, O'Reilly, 2019. 

# Data Source

[US REVIEWS DATASET](https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt)